
# Cleaning pathology data

---

**Author**: Alex Lee

**Institution**: University of Melbourne and Victorian Comprehensive Cancer Centre Alliance

**Date**: 08-03-23 (Updated 19-04-23 to include updated Calcium values)

**Name**: Data cleaning for ten pathology tests in primary care data

**Type**: Pathology tests

**Date range**: 2000 - 2020

**Data sources**:

- Patron (PAT027) primary care database
- NPS MedicineWise primary care database

**Reference**:

- (reference to paper on UWL)

---



## Description

The notebook requires two primary care tables: pathology and patient demographics. These can be from either Patron or NPS, but they need to be structured as shown below.

**Pathology**:

| patientid | result_dte | result_name | result_value | units |
| --- | --- | --- | --- | --- |
| 3 | 23-11-2009 | HAEMOGLOBIN | 200 | mg/L |
| 143 | 09-03-2008 | ALBUMIN | 45 | g/L |

**Patient**:

| usi | patientid | gender_code | year_of_birth | 
| --- | --- | --- | --- |
| 2346 | 3 | Female | 1963 | 
| 766786 | 143 | Male | 1973 | 



The result is a new pathology table that looks like:

| patientid | usi | result_dte | result_name | result_value | units | gender | age | numeric_value | result_value_cleaned | plausible_range | value_range | value_type | units_standard | result_name_standard | 
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 3 | 0001230100 | 23-11-2009 | HAEMOGLOBIN | 200 | g/L | 1956 | Male | 61 | False | 200.0 | True | normal | numeric | g/L | HAEMOGLOBIN | 


## How to run this notebook

The notebook should be run in the following order, based on the section numbers:

1) Define the transformation functions

2) Choose a pathology test

3) Carry out the transformations

4) Check the summary stats to detect any anomalies

5) Write the cleaned data

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

sns.set()
%matplotlib inline


## 1. Data transformation functions

These are to perform slightly more complex data transformations:
- Replacing non-numeric values
- Scaling values by a given factor
- Converting number ranges to median values
- Create range variables
- Create flags based on the type of result value (e.g., a range, numeric or not)

In [2]:
def replace_values(df, column, replacements):
    df['result_value_cleaned'] = df[column]
    for key, value in replacements.items():
        df['result_value_cleaned'] = df.result_value_cleaned.str.replace(key, value, regex=True)
    return df

In [3]:
def create_range_variable(df, range_value_mapping):
    df.loc[:, 'value_range'] = ''
    for value_range, range_description in range_value_mapping.items():
        df.loc[df.result_value_cleaned.between(*value_range), 'value_range'] = range_description
    return df

In [4]:
def range_to_median(df):
    # regex for strings involving ranges of values
    range_patterns = [r'^[0-9]+\s*\-\s*[0-9]+$', 
                      r'^[0-9]+\.[0-9]+\s*\-\s*[0-9]+$', 
                      r'^[0-9]+\s*\-\s*[0-9]+\.[0-9]+$',
                      r'^[0-9]+\.[0-9]+\s*\-\s*[0-9]+\.[0-9]+$']
    
    range_pattern = r'|'.join(range_patterns)

    # check if there are range values
    if df['result_value'].str.contains(range_pattern).sum() == 0:
        return df
    else:
        # subset of data with ranges
        range_mid = (
            df
            .query(f'result_value.str.contains("{range_pattern}")')
            .result_value.str.split('-', expand=True)
            .rename(columns={0: 'value_min', 1: 'value_max'})
            .assign(value_min=lambda df_: df_.value_min.astype(float), 
                    value_max=lambda df_: df_.value_max.astype(float))
            .apply(lambda r: 0.5*(r['value_max'] + r['value_min']), axis=1)
        )
        
        df.loc[range_mid.index, 'result_value_cleaned'] = range_mid
        
        return df

In [5]:
def create_value_type(df, value_to_type):
    df['value_type'] = 'numeric'
    for value, value_type in value_to_type.items():
        df.loc[df['result_value'].str.contains(value, regex=True, na=False), 'value_type'] = value_type
    return df

In [6]:
def map_values(dataset, mapping, column="value_range"):
    """
    Given a dataset and a value mapping, produce a new column that
    takes on values based on those specified in the mapping
    
    Args:
    dataset (pd.DataFrame): the dataset to map values from
    mapping (list of dict): each dict contains inputs and outputs describing
        the mapping
    column (str, default="value_range"): name of new column
        
    Return:
    dataset_updated (pd.DataFrame): dataset with additional column
    """
    dataset[column] = ''
    
    for mapping_element in mapping:
        inputs = mapping_element['input']
        output = mapping_element['output']
        
        # creates the filter to the rows that satisfy all conditions in mapping_element
        condition = pd.Series(index=dataset.index, data=True)
        for key, value in inputs.items():
            if isinstance(value, tuple):
                value_min, value_max = value
                condition = condition & (dataset[key] <= value_max) & (dataset[key] >= value_min)
            else:
                condition = condition & (dataset[key] == value)
        dataset.loc[condition, column] = output
        
    return dataset

In [7]:
def scale_values(pathology, scale_units, scale_factor):
    """
    Scale a subset of the pathology results

    Args:
    pathology (pd.DataFrame): pathology dataset
    scale_units (list of str): specific subset of units to scale values of
    scale_factor (float): amount the multiply the above subset of values by

    Return:
    pathology (pd.DataFrame): dataframe with scaled result values
    """

    pathology.loc[pathology['units'].isin(scale_units), 'result_value_cleaned'] = \
        pathology.query('units in @scale_units')['result_value_cleaned'] * scale_factor

    return pathology

In [8]:
def clean_pathology_data(df, test_data):
    """
    Clean a given pathology data
    
    Args
    ----
    df (pd.DataFrame): input pathology data
    test_data (dict): the meta data used to filter to the appropriate pathology test
    """
    
    test_name = test_data['test_name']
    patterns_test = test_data['test_regex']
    plausible_range = test_data['plausible_range']
    
    scale_units = test_data['scale_units']
    scale_factor = test_data['scale_factor']
    
    units_keep = test_data['units_to_keep']
    units_remove = test_data['units_to_remove']
    units = test_data['units_standard']
    
    range_pattern = test_data['range_pattern']
    value_types = test_data['value_types']
    
    weird_values_replace = test_data['replacement_rules']
    mapping = test_data['mapping']

    pattern_units1 = r'|'.join(units_keep)
    pattern_units2 = r'|'.join(units_remove)
    
    df_cleaned = (
        df
        .assign(result_name=lambda df_: df_.result_name.str.upper().str.strip(), 
                result_value=lambda df_: df_.result_value.str.strip())
        .query(f'result_name.str.strip().str.upper().str.match("{patterns_test}", na=False)')
        .query('result_value.isnull() == False')
        .assign(numeric_value=pd.to_numeric(pathology.result_value, errors='coerce').isnull() == False)
        .query(f'units.str.contains("{pattern_units1}", regex=True, na=True) == True')
        .query(f'units.str.contains("{pattern_units2}", regex=True, na=False) == False')
        .pipe(replace_values, 'result_value', weird_values_replace)
        .pipe(range_to_median)
        .assign(result_value_cleaned=lambda df_: df_.result_value_cleaned.replace(r'^$', np.nan, regex=True))
        .assign(result_value_cleaned=lambda df_: pd.to_numeric(df_.result_value_cleaned, errors='coerce').astype(float))
        .assign(plausible_range=lambda df_: df_.result_value_cleaned.between(*plausible_range))
        .pipe(scale_values, scale_units, scale_factor)
        .pipe(map_values, mapping)
        .pipe(create_value_type, value_types)
        .assign(units_standard=units)
        .assign(result_name_standard=test_name)
    )
    
    return df_cleaned

In [9]:
range_patterns = [r'^[0-9]+\s*\-\s*[0-9]+$', 
                  r'^[0-9]+\.[0-9]+\s*\-\s*[0-9]+$', 
                  r'^[0-9]+\s*\-\s*[0-9]+\.[0-9]+$',
                  r'^[0-9]+\.[0-9]+\s*\-\s*[0-9]+\.[0-9]+$']

range_pattern = r'|'.join(range_patterns)

## 2. Pathlogy test definitions

Each of the 10 pathology tests that we are using are defined below. The key elements of these definitions are:
- The set regular expression patterns to match to the result_name field
- Which records (if any) to remove and which to keep based on their units
- The standard units of the test
- The standard name of the test
- The definitions of reference ranges (e.g., low, normal, elevated)
- Which value ranges are biologically plausible

For this study we have done the cleaning and mapping for the following tests:
- Albumin
- Alkaline Phosphatase
- Bilirubin
- Calcium
- C-Reactive Protein
- Creatinine
- ESR
- Haemoglobin
- Leucocytes
- Platelets

In [10]:
# flags for ranges, inequalities etc
value_types = {r'.[0-9]+\-[0-9]+.': 'range', 
               r'\<.+|<.+|^[0-9]+\-': 'maximum', 
               r'\>.+|>.+|^[0-9]+\+': 'minimum', 
               r'^[^0-9]+$': np.nan}

In [11]:
# covers: replacing inequalities, removing non-numeric strings
weird_values_replace = {r'>\\S\\([0-9.]+)': '\g<1>',
                        r'>\s?([0-9.]+)': '\g<1>',
                        r'>\s*\^([0-9.]+)': '\g<1>',
                        r'>\^([0-9.]+)': '\g<1>',
                        r'>\s*([0-9.]+)': '\g<1>',
                        r'>\s?\^([0-9.]+)': '\g<1>',
                        r'<\\S\\([0-9.]+)': '\g<1>',
                        r'<\s?([0-9.]+)': '\g<1>',
                        r'<\s*\^([0-9.]+)': '\g<1>',
                        r'<\^([0-9.]+)': '\g<1>',
                        r'<\s*([0-9.]+)': '\g<1>',
                        r'<\s?\^([0-9.]+)': '\g<1>',
                        r'^([0-9]+)\+$': '\g<1>',
                        r'^([0-9.]+)\+$': '\g<1>',
                        r'^([0-9]+)\-$': '\g<1>',
                        r'^([0-9.]+)\-$': '\g<1>',
                        r'^([0-9]+)\`$': '\g<1>',
                        r'^([0-9.]+)\`$': '\g<1>',
                        r'^([0-9\.]+)[A-Za-z\.\s]+$': '\g<1>', 
                        r'^[0]*\,([0-9]{2})$': '\g<1>',
                        r'^[^0-9\.]+$': '', 
                        r'([0-9\.]{3}).+': '\g<1>', 
                        r'(^[^0-9\.]+$)': ''}

### Alkaline Phosphatase

In [12]:
# relevant patterns (define the test)
alp_patterns = ['^ALP$', 
                '^ALP (SERUM)$', 
                '.*ALKALINE PHOS.*', 
                '.*ALK PHOS.*', 
                '.*ALK\. PHOS.*', 
                '^ALKALINE PHOS.*', 
                '^ALK PHOS.*',
                '^ALK\. PHOS.*']

alp_regex = r'|'.join(alp_patterns)

In [13]:
alp_mapping = [{'input': {'result_value_cleaned': (0, 30-0.0001)}, 'output': 'low'}, 
               {'input': {'result_value_cleaned': (30, 130)}, 'output': 'normal'},
               {'input': {'result_value_cleaned': (130+0.0001, 180)}, 'output': 'high'},
               {'input': {'result_value_cleaned': (180+0.0001, 10000000)}, 'output': 'abnormal_high'}]

In [14]:
alp_metadata = {'test_name': 'ALP', 
                'test_regex': alp_regex,
                'plausible_range': (30, 150),
                'scale_units': ['U/L'],
                'scale_factor': 1,
                'units_to_keep': ['.+', ''],
                'units_to_remove': [r'^U/g$', r'^U/g Protein$'],
                'units_standard': 'U/L',
                'range_pattern': range_pattern, 
                'value_types': value_types, 
                'replacement_rules': weird_values_replace, 
                'mapping': alp_mapping}

### Albumin

In [13]:
# relevant patterns (define the test)
albumin_patterns = [r'^ALBUMIN$', 
                    r'^ALBUMIN[\s\:\.]+$',
                    r'^S ALBUMIN$', 
                    r'^S ALBUMIN[\s\:\.]$',
                    r'^SERUM ALBUMIN$', 
                    r'^SERUM ALBUMIN[\s\:\.]+$',
                    r'^ALBUMIN[\.\s]+$', 
                    r'^ALBUMIN \(SERUM\)[\s\.\:]+$', 
                    r'^ALBUMIN \(SERUM\)$',
                    r'^ALBUMIN \(CALCULATED\)$', 
                    r'^ALB$']

albumin_regex = r'|'.join(albumin_patterns)

In [14]:
albumin_mapping = [{'input': {'result_value_cleaned': (0, 35-0.001)}, 'output': 'low'}, 
                   {'input': {'result_value_cleaned': (35, 50)}, 'output': 'normal'},
                   {'input': {'result_value_cleaned': (50+0.001, 100)}, 'output': 'high'},
                   {'input': {'result_value_cleaned': (100+0.001, 10000000)}, 'output': 'abnormal_high'}]

In [15]:
albumin_metadata = {'test_name': 'ALBUMIN', 
                    'test_regex': albumin_regex,
                    'plausible_range': (0, 100),
                    'scale_units': ['g/L'],
                    'scale_factor': 1,
                    'units_to_keep': ['.+', ''],
                    'units_to_remove': ['mg/L', 'ug/ml'],
                    'units_standard': 'g/L',
                    'range_pattern': range_pattern, 
                    'value_types': value_types, 
                    'replacement_rules': weird_values_replace, 
                    'mapping': albumin_mapping}

### Bilirubin

In [16]:
# relevant patterns (define the test)
bilirubin_patterns = ['^TOTAL BILIRUBIN$', 
            r'^TOTAL BILIRUBIN[\s\:]+$',
            '^BILIRUBIN \(TOTAL\)$', 
            '^S BILIRUBIN[\s\:]+$', 
            '^S BILIRUBIN$', 
            '^BILIRUBIN\,TOTAL$', 
            '^TOTALBILI.*', 
            '^BILIRUBIN$', 
            '^BILIRUBIN \- TOTAL$', 
            '^SERUM BILIRUBIN$', 
            '^BILIRUBIN TOTAL$',
            '^BILIRUBIN[\s\.\:]+', 
            '^P TOTAL BILIRUBIN$', 
            'BILIRUBIN TOTAL \(ADULT\)']

bilirubin_regex = r'|'.join(bilirubin_patterns)

In [17]:
bilirubin_mapping = [{'input': {'result_value_cleaned': (0, 3-0.001)}, 'output': 'low'}, 
           {'input': {'result_value_cleaned': (3, 17+0.001)}, 'output': 'normal'},
           {'input': {'result_value_cleaned': (17 +0.001, 100)}, 'output': 'high'},
           {'input': {'result_value_cleaned': (100.00001, 10000000)}, 'output': 'abnormal_high'}]

In [18]:
bilirubin_metadata = {'test_name': 'BILIRUBIN', 
                      'test_regex': bilirubin_regex,
                      'plausible_range': (0.1, 25),
                      'scale_units': ['umol/L'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': ['----'],
                      'units_standard': 'umol/L',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': bilirubin_mapping}

### Calcium

In [19]:
# relevant patterns (define the test)
calcium_patterns = [r'^CALCIUM$',
            r'^CALCIUM[\s\:\.]+$',
            r'^TOTAL CALCIUM$', 
            r'^TOTAL CALCIUM[\s\:\.]+$',
            r'^SERUM CALCIUM$', 
            r'^SERUM CALCIUM[\s\:\.]+$',
            r'^CALCIUM CONCENTRATION$', 
            r'^S CALCIUM$',
            r'^S CALCIUM[\s\:\.]+$',
            r'^CALCIUM \(TOTAL\)$', 
            r'^CALCIUM\, TOTAL$', 
            r'^CALCIUM SERPL\-SCNC$', 
            r'^CALCIUM \(CAL\.\)$', 
            r'^CALCIUM LEVEL[\s\:\.]+$',
            r'^CALCIUM[\s\.]+$', 
            r'^CALCIUM SERUM$',  
            r'^CALCIUM SERUM[\s\:\.]+$', 
            r'T\-CALCIUM', 
            r'^CALCIUM CONC\.$', 
            r'^CALCIUM TOTAL$', 
            r'^P CALCIUM$', 
            r'^CALCIUM[\s\.]+$', 
            r'^ADJUSTED CA.*$', 
            r'^CA \(CORR\)$', 
            r'^CA\(CORRECTED\)$', 
            r'^CALC ALB CORR.*$', 
            r'^CALCIUM \(ADJUSTED\)$', 
            r'CALCIUM \(ALB.* ADJ.*\)$', 
            r'CALCIUM \(ALB\. CORR\.\)$', 
            r'^CALCIUM \(CORRECTED\)$', 
            r'^CALCIUM\.ADJUSTED$', 
            r'^CALCIUM CORR.*$', r'^CALCIUM\-CORR.*$', r'^CORR.* CA.*$', r'^S CALCIUM \(CORRECTED\)$']

calcium_regex = r'|'.join(calcium_patterns)

In [20]:
calcium_mapping = [{'input': {'result_value_cleaned': (0, 2.12-0.0001)}, 'output': 'low'}, 
                   {'input': {'result_value_cleaned': (2.12, 2.6)}, 'output': 'normal'},
                   {'input': {'result_value_cleaned': (2.6+0.00001, 5.0)}, 'output': 'high'},
                   {'input': {'result_value_cleaned': (5, 10000000)}, 'output': 'abnormal_high'}]

In [21]:
calcium_metadata = {'test_name': 'CALCIUM', 
                      'test_regex': calcium_regex,
                      'plausible_range': (1, 5),
                      'scale_units': ['mmol/L'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': [r'mmol/day', r'mmol/d', r'mmol/24h'],
                      'units_standard': 'mmol/L',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': calcium_mapping}

### C-Reactive Protein

In [22]:
# relevant patterns (define the test)
crp_patterns = ['.*CRP.*', 
            'C REACTIVE PROTEIN.*', 
            'C\-REACTIVE PROTEIN.*', 
            'S C\-REACTIVE PROTEIN.*', 
            'SERUM C\-REACTIVE PROTEIN.*']

crp_regex = r'|'.join(crp_patterns)

In [23]:
crp_mapping = [{'input': {'result_value_cleaned': (10+0.0001, 2000)}, 'output': 'elevated'}, 
               {'input': {'result_value_cleaned': (0, 10)}, 'output': 'normal'}]

In [24]:
crp_metadata = {'test_name': 'CRP', 
                      'test_regex': crp_regex,
                      'plausible_range': (0, 2000),
                      'scale_units': ['mg/L'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': ['^g/L', '.?U/L'],
                      'units_standard': 'mg/L',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': crp_mapping}

### Creatinine

In [25]:
# relevant patterns (define the test)
creatinine_patterns = [r'^CREATININE$', 
            r'^CREATININE CONCENTRATION$', 
            r'^CREATININE CONCENTRATION[\s\:]+$', 
            r'^SERUM CREATININE$', 
            r'^SERUM CREATININE[\s\:]+$',
            r'^S CREATININE$', 
            r'^S CREATININE[\s\:]+$',
            r'^CREATININE (SERUM)$'
            r'CREATININE \(BLOOD\)', 
            r'^CREATININE \(UMOL\/L\)$', 
            r'^CREATININE POCT$', 
            r'^CREATININE[\s\.]+$', 
            r'CREATININE \(IDMS\)', 
            r'^CREATININE \- POCT$', 
            r'^CREATININE \(SERUM\/PLASMA\; MOLES\/VOLUME\)$', 
            r'^CREATININE\-I$', 
            r'^CREATININE \(MMOL\/L\)$', 
            r'^CREATININE[\s\.\:]+$', 
            r'^P CREATININE$', 
            r'^SERUM CREAT\.{0,1}$']

creatinine_regex = r'|'.join(creatinine_patterns)

In [26]:
# general specification of subset of data -> range mapping
# assume only age, sex, other ordinal value, result_value
creatinine_mapping = [{'input': {'gender': 'Male', 'result_value_cleaned': (60, 106)}, 'output': 'normal'}, 
           {'input': {'gender': 'Male', 'result_value_cleaned': (0, 60-0.0001)}, 'output': 'low'}, 
           {'input': {'gender': 'Male', 'result_value_cleaned': (106+0.0001, 100000)}, 'output': 'high'}, 
           {'input': {'gender': 'Female', 'result_value_cleaned': (45, 80)}, 'output': 'normal'}, 
           {'input': {'gender': 'Female', 'result_value_cleaned': (0, 45-0.001)}, 'output': 'low'},
           {'input': {'gender': 'Female', 'result_value_cleaned': (80+0.001, 100000)}, 'output': 'high'}]

creatinine_metadata = {'test_name': 'CREATININE', 
                      'test_regex': creatinine_regex,
                      'plausible_range': (0.1, 200),
                      'scale_units': ['umol/L'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': [r'^mmol.+$', r'^mg.+$'],
                      'units_standard': 'umol/L',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': creatinine_mapping}

### ESR

In [27]:
# relevant patterns (define the test)
esr_patterns = ['ERYTHROCYTE SEDIMENTATION RATE', 
            'ERYTHROCYTE SED RATE', 
            '^ESR.*', 
            '^E\.S\.R.*']

esr_regex = r'|'.join(esr_patterns)

In [28]:
# general specification of subset of data -> range mapping
# assume only age, sex, other ordinal value, result_value
esr_mapping = [{'input': {'gender': 'Male', 'age': (0, 65), 'result_value_cleaned': (0, 10)}, 'output': 'normal'}, 
           {'input': {'gender': 'Male', 'age': (0, 65), 'result_value_cleaned': (10+0.0001, 10000)}, 'output': 'high'}, 
           {'input': {'gender': 'Male', 'age': (65, 130), 'result_value_cleaned': (0, 20)}, 'output': 'normal'}, 
           {'input': {'gender': 'Male', 'age': (65, 130), 'result_value_cleaned': (20+0.0001, 10000)}, 'output': 'high'},
           {'input': {'gender': 'Female', 'age': (0, 65), 'result_value_cleaned': (0, 15)}, 'output': 'normal'}, 
           {'input': {'gender': 'Female', 'age': (0, 65), 'result_value_cleaned': (15+0.0001, 10000)}, 'output': 'high'}, 
           {'input': {'gender': 'Female', 'age': (65, 130), 'result_value_cleaned': (0, 25)}, 'output': 'normal'}, 
           {'input': {'gender': 'Female', 'age': (65, 130), 'result_value_cleaned': (25+0.0001, 10000)}, 'output': 'high'}]

esr_metadata = {'test_name': 'ESR', 
                      'test_regex': esr_regex,
                      'plausible_range': (0, 200),
                      'scale_units': ['mm/H'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': ['----'],
                      'units_standard': 'mm/H',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': esr_mapping}

### Haemoglobin

In [29]:
haemoglobin_patterns = ['^HAEMAGLOBIN$', 
                        '^HAEMOGLOBIN$', 
                        '^HAEMOGLOBIN \(CALC\)$', 
                        '^HAEMOGLOBIN \(GEM\)$', 
                        '^HAEMOGLOBIN\s{0,1}[\.\s:;]+$', 
                        '^HAEMOGLOBIN;TOTAL$', 
                        '^HB$', 
                        '^HEMOGLOBIN$',
                        '^HGB$', 
                        '^TOTAL HAEMOGLOBIN$', 
                        '^TOTAL HB$']

haemoglobin_regex = r'|'.join(haemoglobin_patterns)

In [30]:
pattern_units1 = r'|'.join(['.+', ''])
pattern_units2 = r'|'.join(['mg/L', 'ug/ml'])

In [33]:
# general specification of subset of data -> range mapping
# assume only age, sex, other ordinal value, result_value
haemoglobin_mapping = [{'input': {'gender': 'Male', 'result_value_cleaned': (130, 180)}, 'output': 'normal'}, 
           {'input': {'gender': 'Male', 'result_value_cleaned': (0, 130-0.0001)}, 'output': 'low'}, 
           {'input': {'gender': 'Male', 'result_value_cleaned': (180 + 0.0001, 100000)}, 'output': 'high'}, 
           {'input': {'gender': 'Female', 'result_value_cleaned': (115, 160)}, 'output': 'normal'}, 
           {'input': {'gender': 'Female', 'result_value_cleaned': (0, 115-0.0001)}, 'output': 'low'},
           {'input': {'gender': 'Female', 'result_value_cleaned': (160+0.0001, 100000)}, 'output': 'high'}]

haemoglobin_metadata = {'test_name': 'HAEMOGLOBIN', 
                      'test_regex': haemoglobin_regex,
                      'plausible_range': (30, 250),
                      'scale_units': ['g/dl', 'g/dL'],
                      'scale_factor': 10,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': ['mg/L', 'ug/ml'],
                      'units_standard': 'g/L',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': haemoglobin_mapping}

### Leucocytes

In [34]:
# relevant patterns (define the test)
patterns = [r'.*LEUCOCYTE COUNT.*', 
            r'^LEUCOCYTES$', 
            r'^LEUKOCYTES$', 
            r'LEUC'
            r'^WBC$', 
            r'^WCC$', 
            r'WHITE CELL', 
            r'^WHITECELLCOUNT$', 
            r'^WHITE CELL']

leucocytes_regex = r'|'.join(patterns)

In [35]:
# general specification of subset of data -> range mapping
# assume only age, sex, other ordinal value, result_value
leucocytes_mapping = [{'input': {'result_value_cleaned': (0, 4.0-0.0001)}, 'output': 'low'}, 
           {'input': {'result_value_cleaned': (4.0, 11)}, 'output': 'normal'}, 
           {'input': {'result_value_cleaned': (11+0.0001, 25)}, 'output': 'high'}, 
           {'input': {'result_value_cleaned': (25+0.0001, 1000000000)}, 'output': 'abnormal_high'}]

leucocytes_metadata = {'test_name': 'LEUCOCYTES', 
                      'test_regex': leucocytes_regex,
                      'plausible_range': (0, 100),
                      'scale_units': ['nL'],
                      'scale_factor': 1,
                      'units_to_keep': ['9', 'nL'],
                      'units_to_remove': ['----'],
                      'units_standard': 'nL',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': leucocytes_mapping}

### Platelets

In [36]:
# relevant patterns (define the test)
platelets_patterns = ['PLATELET COUNT', 
                      'PLATELET COUNT[\s\.\;\:]$', 
                      'PLATELETCOUNT', 
                      '^PLATELETS[\s\.\:\;]+$', 
                      '^PLATELETS$', # modify regex
                      '^PLATELET$', 
                      '^PLAT$', 
                      '^PLT$', 
                      '^PLTI$', 
                      '^PLTS$',
                      '^PLAT COUNT$', 
                      '^PLATELET \# BLD AUTO$', 
                      '^[\s\.]+PLATELET.']

platelets_regex = r'|'.join(platelets_patterns)

In [37]:
# general specification of subset of data -> range mapping
# assume only age, sex, other ordinal value, result_value
platelets_mapping = [{'input': {'result_value_cleaned': (0, 149.9999)}, 'output': 'low'}, 
           {'input': {'result_value_cleaned': (150, 400)}, 'output': 'normal'}, 
           {'input': {'result_value_cleaned': (400.00001, 1000000000)}, 'output': 'high'}]

platelets_metadata = {'test_name': 'PLATELETS', 
                      'test_regex': platelets_regex,
                      'plausible_range': (10, 1000),
                      'scale_units': ['nL'],
                      'scale_factor': 1,
                      'units_to_keep': ['.+', ''],
                      'units_to_remove': ['----'],
                      'units_standard': 'nL',
                      'range_pattern': range_pattern, 
                      'value_types': value_types, 
                      'replacement_rules': weird_values_replace, 
                      'mapping': platelets_mapping}

## 3. Load data

In [38]:
# for NPS data
source_folder1 = "M:/Working/DataAnalysis/CleanAndStructure/DataFiles"

# for Patron data
source_folder = "M:/Working/AL/projects/weight_loss/outputs/patron_data_extracts"

In [39]:
# for NPS data
filename_pathology = "NPS_PathologyResults_202005.csv"
filename_patient = "NPS_Patient_202107.csv"

# for Patron data
#filename_pathology = "patron_pathology_data_310123.csv"
#filename_pathology_pat12 = "PAT012_ivx_idvl_tst.parquet"
#filename_patient = "PAT012_pat_pat_dtl.parquet"

In [ ]:
# nps
pathology = pd.read_csv(f'{source_folder1}/{filename_pathology}')
patient = pd.read_csv(f'{source_folder1}/{filename_patient}')

# patron
#pathology = pd.read_parquet(f'{source_folder1}/{filename_pathology_pat12}')
#patient = pd.read_parquet(f'{source_folder1}/{filename_patient}')

In [41]:
# only required for patron
patient.query('Source_System == "ZM"').Patient_USI.to_csv('M:/Working/DataAnalysis/CleanAndStructure/PAT012_ZM_USI.csv', index=False)

In [42]:
# this is only needed for patron data
pathology = (
    pathology
    .rename(columns={'Patient_PPN': 'patientid', 
                     'Patient_USI': 'usi', 
                     'Age_at_Event': 'age', 
                     'Result_Date': 'result_dte', 
                     'Result_Name': 'result_name', 
                     'Result_Value': 'result_value', 
                     'Result_Units': 'units', 
                     'LOINC_Code': 'loinc_code', 
                     'Source_System': 'source_system'})
)

patient = (
    patient
    .rename(columns={'Patient_PPN': 'patientid', 
                     'Patient_USI': 'usi', 
                     'YearOfBirth': 'year_of_birth', 
                     'PATRON_Gender_Lkp': 'gender_code'})
)

In [41]:
# for NPS
# mapping between patientid and gender
patientid_to_gender = dict(
    patient[['patientid', 'gender_code']]
    .drop_duplicates()
    .groupby('patientid', as_index=False)
    .first()
    .values
)

# mapping between patientid and year of birth
patientid_to_yob = dict(
    patient[['patientid', 'year_of_birth']]
    .drop_duplicates()
    .groupby('patientid', as_index=False)
    .first()
    .values
)

# add gender and year of birth columns
pathology = (
    pathology
    .assign(yob=lambda df_: df_.patientid.map(patientid_to_yob),
            gender=lambda df_: df_.patientid.map(patientid_to_gender),
            result_dte=lambda df_: pd.to_datetime(df_.result_dte),
            age=lambda df_: df_.result_dte.dt.year - df_.yob)
)

## 4. Transform the data

The following steps
- Filter to pathology result names that corrrespond to the required test
- Remove rows with missing values
- Create a flag for whether column is numeric or contains other characters
- Remove rows with incorrect units
- Clean the non-numeric values based on specific rules from regular expressions
- Convert the 'range values', i.e., values such as '5-10' to their median
- Replace empty strings with nan
- Convert the cleaned result values to float
- Create a column that flags those values that are and aren't biologically plausible
- Create a column that categorises values into different normal / abnormal categories
- Create a column for the type of value that is in the result, e.g., a range, a max, a min
- Create a column for the unit (so there is only one way of describing the units)
- Create a column for the standard path test name

In [42]:
pathology_metadata_list = [albumin_metadata, 
                           alp_metadata, 
                           bilirubin_metadata, 
                           calcium_metadata,
                           creatinine_metadata, 
                           crp_metadata, 
                           esr_metadata, 
                           haemoglobin_metadata, 
                           leucocytes_metadata, 
                           platelets_metadata]

In [43]:
output_folder = "M:/Working/AL/Projects/weight_loss/outputs"

In [ ]:
for metadata in pathology_metadata_list[4:]:
    print(f"Cleaning data for {metadata['test_name']}")
    pathology_cleaned = clean_pathology_data(pathology, metadata)
    pathology_cleaned.to_parquet(f'{output_folder}/pathology_nps_{metadata["test_name"].lower()}_cleaned_Vic_ranges_200923.parquet')


## 5. Basic summary stats

For diagnosing issues with the data transformations

In [203]:
df = pd.read_parquet(f'{output_folder}/pathology_patron_creatinine_cleaned_BN_ranges_130423.parquet')

In [212]:
test_values2 = df.query('units.isnull() == False').result_value_cleaned.values

In [2]:
df.shape[0] - df_temp.shape[0]

In [171]:
df_temp = df.query('units.isnull()')

In [1]:
df_temp.loinc_code.value_counts()

## 6. Write the cleaned data

In [35]:
output_folder = 'M:/Working/AL/projects/weight_loss/outputs'

In [37]:
pathology_cleaned.to_parquet(f'{output_folder}/pathology_nps_platelets_cleaned_130323.parquet')